In [1]:
from __future__ import division
import time
import numpy as np
import pandas as pd
import pymanopt as pm
import scipy.io as sio
import scipy.linalg as la
from sklearn.svm import SVC, LinearSVC
import autograd.numpy as anp
import pymanopt.solvers as solvers
import pymanopt.manifolds as manifolds
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA

In [2]:
class JDME_MMD:
    def __init__(self,kernel='linear',estimator=SVC(kernel='linear'),dimension=100,
                 degree=2,sigma=None,T=10,maxiter=5,verbosity=0,verb=False,preprocessing=True):    
        args_values = locals()                    
        args_values.pop("self")                   
        for arg,value in args_values.items():    
            setattr(self,arg,value)
    
    def _fit(self,Xs,ys,Xt,yt0,W0=None):         
                                                 
        ns,dim = Xs.shape
        nt = Xt.shape[0]
        n = ns + nt
        X = np.vstack((Xs,Xt))
                
        # construct MMD matrix
        e =  np.hstack((1. / ns * np.ones(ns),- 1. / nt * np.ones(nt)))
        L = np.outer(e,e)
                        
        if self.sigma is None:
            pairwise_dist = pairwise_distances(X,X,'sqeuclidean')       
            self.sigma = np.median(pairwise_dist[pairwise_dist!=0])     
        
        # construct the label kernel matrix                             
        y = np.hstack((ys,yt0))                                     
        UniCls = np.unique(y)
        Ky = np.ones((n,n))*(y[:,None] == y) 
        for eachCls in UniCls:
            idx_s = np.where(ys==eachCls)
            idx_t = np.where(yt0==eachCls)
            nt_c = len(idx_t[0])
            ns_c = len(idx_s[0])
            if nt_c == 0 :
                e = np.hstack(((ns/ns_c) * np.ones(ns),  np.zeros(nt)))
            else:
                e = np.hstack(((ns/ns_c) * np.ones(ns),  (nt/nt_c) * np.ones(nt)))
            L2 = np.outer(e,e)
            tmp = y == eachCls
            idx = tmp[:,None]*tmp
            L2 = L2 * idx
            Ky = Ky + L2           
                
        # construct the objective function 
        def objW(W):
            """
            code from 
            https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python
            """
            WX = anp.dot(X,W)
            if self.kernel == 'linear':
                K_W = anp.dot(WX,WX.T) * Ky  
                MMD = anp.trace(anp.dot(K_W,L))
            elif self.kernel == 'poly':
                K_W = ( anp.dot(WX,WX.T) + 1.) ** self.degree * Ky  
                MMD = anp.trace(anp.dot(K_W,L))
            elif self.kernel == 'rbf':
                WX_norm = anp.sum(WX ** 2, axis = -1)
                K_W = anp.exp(-(WX_norm[:,None] + WX_norm[None,:] - 2 * anp.dot(WX, WX.T)) / self.sigma) * Ky                
                MMD = anp.trace(anp.dot(K_W,L))
            else:
                pass
            return MMD

        manifold = manifolds.Grassmann(dim, self.dimension)                          
        problem = pm.Problem(manifold=manifold, cost=objW,verbosity=self.verbosity)  
                                                                                     
        if W0 is None:
            W = solvers.SteepestDescent(maxiter=self.maxiter).solve(problem) 
        else:
            W = solvers.SteepestDescent(maxiter=self.maxiter).solve(problem,x=W0[:,:self.dimension])  
        return W

    def fit(self,Xs,ys,Xt,yt=None):
        """
        here yt is only used to show the trend of the accuracy during iteration. 
        """
        yt0 = self.estimator.fit(Xs,ys).predict(Xt)             
        W0 = PCA().fit(np.vstack((Xs,Xt))).components_.T     
        for i in range(self.T):            
            W = self._fit(Xs,ys,Xt,yt0,W0)
            Xs_reduced,Xt_reduced = Xs.dot(W),Xt.dot(W)
            
            if True == self.preprocessing:
                Xs_reduced = Xs_reduced / la.norm(Xs_reduced,axis=1,keepdims=True)
                Xt_reduced = Xt_reduced / la.norm(Xt_reduced,axis=1,keepdims=True)
            yt0 = self.estimator.fit(Xs_reduced,ys).predict(Xt_reduced)
            W0 = W
            if self.verb == True:
                print(str(i+1)+'th iter. acc. :',self.estimator.score(Xt_reduced,yt))
        self.W = W
        Xs = Xs.dot(W)
        Xs = Xs / la.norm(Xs,axis=1,keepdims=True)
        self.estimator.fit(Xs,ys)
        return self
    
    def predict(self,Xt):
        Xt = Xt.dot(self.W)
        Xt = Xt / la.norm(Xt,axis=1,keepdims=True)
        yt = self.estimator.predict(Xt)
        return yt
   
    def predict_score(self,Xt,yt):
        Xt = Xt.dot(self.W)
        Xt = Xt / la.norm(Xt,axis=1,keepdims=True)
        score = self.estimator.score(Xt,yt)
        return score

In [3]:
def readData(sourcePath,targetPath):
    sourceData = sio.loadmat(sourcePath)
    targetData = sio.loadmat(targetPath)
    Xs,ys = sourceData['features'].T.astype(np.float64),sourceData['labels'].ravel().astype(np.float64)  
    Xt,yt = targetData['features'].T.astype(np.float64),targetData['labels'].ravel().astype(np.float64)  
    Xs = Xs / la.norm(Xs,axis=1,keepdims=True) 
    Xt = Xt / la.norm(Xt,axis=1,keepdims=True) 
    return Xs,ys,Xt,yt

In [4]:
estimator = SVC(kernel='linear')

In [5]:
dataSetPath = '../dataset/COIL/'
domains = ['C1','C2']
domcouples = []
res = []
for sc in domains:
    for tg in domains:
        if sc != tg:
            start = time.time()
            sourcePath = dataSetPath + sc + '.mat'
            targetPath = dataSetPath + tg + '.mat'
            Xs,ys,Xt,yt = readData(sourcePath,targetPath)
            score = JDME_MMD(dimension=200,kernel='poly',estimator=estimator,T=20).fit(Xs,ys,Xt).predict_score(Xt,yt)
            domcouples.append(sc + '->' + tg)
            res.append(score)
            print('%s : %.2f, \t time : %.2fs'%(domcouples[-1],res[-1]*100,time.time()-start))
result = pd.DataFrame(data=res,index=domcouples, columns=['JDME-SVM'])
print('mean score:  '+str(result.values.mean()))
result

C1->C2 : 91.39, 	 time : 208.91s
C2->C1 : 93.06, 	 time : 204.13s
mean score:  0.9222222222222223


,JDME-SVM
C1->C2,0.913889
C2->C1,0.930556
